# Load the Essay Data

In [9]:
from BrattEssay import load_bratt_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

import pickle

import torch
from torch import nn
from torch import autograd, optim
import torch.nn.functional as F

In [7]:
settings = Settings()

root_folder      = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder  = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [8]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

## Prepare Tags

In [10]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))
vtags = set(regular_tags)
vtags.add(EMPTY_TAG)

len(unique_words)

1641

## Words To Ids

In [45]:
from collections import defaultdict

word_to_ix = {}
tag_to_ix = {}

#pre-compute id sequences so we don't have to remap each time
#xs
wd_id_seqs  = []
#ys
tag_id_seqs = []
# for computing accuracy
tag_seqs    = []

START = "<start>"
END   = "<end>"

for essay in tagged_essays:
    for sentence in essay.sentences:
        
        wd_ids = []
        tag_ids = []
        tag_seq = []
        
        wd_id_seqs.append(wd_ids)
        tag_id_seqs.append(tag_ids)
        tag_seqs.append(tag_seq)
        
        for word, tags in [(START, set())] + sentence + [(END, set())]:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
            wd_ids.append(word_to_ix[word])
            
            tags = vtags.intersection(tags)
            if len(tags) > 1:
                tag = max(tags, key=lambda t: tag_freq[t])                
            else:
                tag = EMPTY_TAG
            if tag not in tag_to_ix:
                tag_to_ix[tag] = len(tag_to_ix)
            tag_ids.append(tag_to_ix[tag])
            tag_seq.append(tag)
            
# invert the dictionary
ix_to_tag  = dict(zip(tag_to_ix.values(),  tag_to_ix.keys()))
ix_to_word = dict(zip(word_to_ix.values(), word_to_ix.keys()))

len(tag_id_seqs), len(wd_id_seqs)
wd_seq_lens  = np.asarray(list(map(len, wd_id_seqs)))
tag_seq_lens = np.asarray(list(map(len, tag_id_seqs)))
assert set(wd_seq_lens == tag_seq_lens) == set([True]), "Sequence lengths must match"

## Train\Test Split

In [22]:
#20% split
test_split = 0.2
train_size = int((1-test_split) * len(wd_id_seqs))

train_xs,   test_xs   = wd_id_seqs[:train_size],  wd_id_seqs[train_size:]
train_tags, test_tags = tag_id_seqs[:train_size], tag_id_seqs[train_size:]
len(train_xs), len(test_xs), len(train_tags), len(test_tags)

(6633, 1659, 6633, 1659)

## Model

In [11]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 100
HIDDEN_DIM = 128

In [23]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.Tensor(1, 1, self.hidden_dim)),
                autograd.Variable(torch.Tensor(1, 1, self.hidden_dim)))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

## To and From Ids

In [46]:
def prepare_id_sequence(id_seq):
    tensor = torch.LongTensor(id_seq)
    return autograd.Variable(tensor)

def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w], seq))
    return prepare_id_sequence(idxs, to_ix)

def argmax(vec):
    # return the argmax as a list of python ints
    _, ixs = torch.max(vec, 1)
    return ixs.view(-1).data.tolist()

def map_to(vals, to_other_val):
    return list(map(lambda v: to_other_val[v], vals))

def to_tags(scores, to_tag):
    ids = argmax(scores)
    return map_to(ids, to_tag)

to_tags(tag_scores, ix_to_tag)

['Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty']

In [63]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [64]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
#inputs = prepare_sequence(train_xs[0], word_to_ix)
inputs = prepare_id_sequence(train_xs[0])
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
-1.5794 -1.5555 -1.6675 -1.6013 -1.6479
-1.5386 -1.5978 -1.7189 -1.6167 -1.5839
-1.5793 -1.5543 -1.7603 -1.6261 -1.5425
-1.5910 -1.6069 -1.6120 -1.7206 -1.5263
-1.5610 -1.6532 -1.5819 -1.7082 -1.5519
-1.5518 -1.6337 -1.6723 -1.6566 -1.5403
-1.6097 -1.5640 -1.7140 -1.5439 -1.6242
-1.6030 -1.6098 -1.5694 -1.6509 -1.6159
-1.6333 -1.5828 -1.5434 -1.6430 -1.6488
-1.5716 -1.6698 -1.5252 -1.7354 -1.5602
-1.5349 -1.6556 -1.4689 -1.7087 -1.7023
-1.5445 -1.6405 -1.4832 -1.7611 -1.6402
-1.5610 -1.6171 -1.5855 -1.6137 -1.6735
[torch.FloatTensor of size 13x5]



In [ ]:
%%time
from numpy.random import shuffle
#TODO - shuffle on each epoch
#TODO - prepare the sequences once only
for epoch in range(100): # again, normally you would NOT do 300 epochs, it is toy data
    print("Epoch={epoch}".format(epoch=epoch))
    ixs = list(range(len(train_xs)))
    # shuffle order of sequences
    shuffle(ixs)
    for ix in ixs:
        sentence, tags = train_xs[ix], train_tags[ix]
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
    
        # Step 2. Get our inputs ready for the network, that is, turn them into Variables
        # of word indices.
        sentence_in = prepare_id_sequence(sentence)
        targets = prepare_id_sequence(tags)
    
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

Epoch=0
Epoch=1


In [49]:
# See what the scores are after training
inputs = prepare_id_sequence(train_xs[0])
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
-3.4429e-05 -1.1487e+01 -1.1858e+01 -1.1737e+01 -1.1609e+01
-2.9907e-05 -1.1663e+01 -1.1913e+01 -1.1946e+01 -1.1722e+01
-9.8127e-06 -1.2718e+01 -1.3139e+01 -1.3063e+01 -1.2812e+01
-4.5709e-06 -1.3482e+01 -1.3837e+01 -1.3908e+01 -1.3564e+01
-4.1281e-06 -1.3638e+01 -1.3932e+01 -1.3957e+01 -1.3654e+01
-2.2053e-06 -1.4173e+01 -1.4577e+01 -1.4622e+01 -1.4339e+01
-2.3518e-05 -1.1820e+01 -1.2360e+01 -1.2315e+01 -1.1815e+01
-1.0255e-05 -1.2685e+01 -1.3148e+01 -1.3045e+01 -1.2701e+01
-1.1635e-04 -1.0442e+01 -1.0636e+01 -1.0638e+01 -1.0148e+01
-1.5191e-04 -9.8049e+00 -1.0487e+01 -1.0449e+01 -1.0130e+01
-2.5109e-05 -1.1695e+01 -1.2250e+01 -1.2296e+01 -1.1812e+01
-3.4449e-06 -1.3621e+01 -1.4161e+01 -1.4170e+01 -1.4014e+01
-3.6310e-06 -1.3664e+01 -1.4031e+01 -1.4096e+01 -1.3915e+01
[torch.FloatTensor of size 13x5]



In [60]:
ix = 10
inputs = prepare_id_sequence(train_xs[ix])
tag_scores = model(inputs)

list(zip(map_to(train_xs[ix],ix_to_word),to_tags(tag_scores, ix_to_tag)))

[('<start>', 'Empty'),
 ('what', 'Empty'),
 ('leads', 'Empty'),
 ('to', 'Empty'),
 ('different', 'Empty'),
 ('in', 'Empty'),
 ('the', 'Empty'),
 ('rates', 'Empty'),
 ('of', 'Empty'),
 ('coral', 'Empty'),
 ('bleaching', 'Empty'),
 ('is', 'Empty'),
 ('that', 'Empty'),
 ('INFREQUENT', 'Empty'),
 ('different', 'Empty'),
 ('types', 'Empty'),
 ('of', 'Empty'),
 ('INFREQUENT', 'Empty'),
 ('living', 'Empty'),
 ('within', 'Empty'),
 ('the', 'Empty'),
 ('coral', 'Empty'),
 ('polyps', 'Empty'),
 ('gives', 'Empty'),
 ('the', 'Empty'),
 ('coral', 'Empty'),
 ('their', 'Empty'),
 ('varying', 'Empty'),
 ('colors', 'Empty'),
 ('.', 'Empty'),
 ('<end>', 'Empty')]

In [42]:
from collections import defaultdict

def to_ys_by_code(seq_tags):
    ys_by_tag = defaultdict(list)
    for seq in seq_tags:
        for tag in seq:
            ys_by_tag[tag].append(1)
            for vtag in vtags:
                if tag != vtag:
                    ys_by_tag.append(0)
    return ys_by_tag

## TODO - prepare the sequences as Tensors once only
## TODO - use a better optimizer - ADAM maybe?